In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.eval()

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

In [212]:
def clear_gpu():
    import gc, torch
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

clear_gpu()


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

# Classification

In [4]:
import json
file_path = "/kaggle/input/classification/classification_instruction_data_en.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    data = [json.loads(line.strip()) for line in f]

In [5]:
valid_labels = {"negative", "positive", "neutral"}
data = [sample for sample in data if sample['output'].strip().lower() in valid_labels]

In [6]:
import re

def extract_answer(text):
    matches = re.findall(r'= *\$?(-?\d+(?:\.\d+)?)', text)
    if matches:
        return matches[-1]  # lấy phép tính cuối cùng có dạng = xxx
    return None

In [7]:
def extract_target(text):
  for line in text.splitlines()[::-1]:
    if line.strip().startswith("####"):
      return line.strip().replace("####","").strip()
  return None

In [8]:
def split_cot_and_answer(output_text):
    lines = output_text.strip().splitlines()
    cot_lines = [line for line in lines if not line.strip().startswith("####")]
    answer = extract_target(output_text)
    return "\n".join(cot_lines), answer

**Zero-shot**

In [9]:
acc = 0
results = []
for i in range(200):
  zero_shot_prompt = f"""
  You are a classifier assistant.{data[i]['instruction']}. Recorrect each time you classifying with other LLM model
  Q: {data[i]['input'] } A: The answer is ?"""
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(data[i]['output'])
  print(de)
  if de == data[i]['output']:
    acc = acc+1
print(acc/200)

neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
positive
positive
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
negative
negative
negative
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
The answer is ?
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutr

Zero-shot + CoT (ReAct)

In [13]:
acc = 0
results = []
for i in range(200):
  zero_shot_prompt = f"""
  You are a classifier assistant.{data[i]['instruction']}.
  Q: {data[i]['input'] } A: Let's think step by step. The answer is ?"""
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(data[i]['output'])
  print(de)
  if de == data[i]['output']:
    acc = acc+1
print('Accuracy', acc/200)

neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
negative
negative
negative
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral

In [11]:
acc = 0
results = []
for i in range(200):
  zero_shot_prompt = f"""
  You are a classifier assistant.{data[i]['instruction']}. Recorrect each time you classifying with other LLM model
  Q: {data[i]['input'] } A: Let's think step by step. The sentiment is """
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de == data[i]['output']:
    acc = acc+1
print(acc/200)

0.8


Zero-shot + ToT

In [84]:
def zero_shot_ToT(text):
    return (
        f"{text['instruction']}. Q: {text['input']}.\n"
        f"A: Let's think of three strategies for this question.\n"
        f"Strategy 1: "
    )

In [43]:
import re

def extract_labels_from_thoughts(text):
    matches = re.findall(r'\b(positive|negative|neutral)\b', text.lower())
    return list(set(matches))  # loại trùng nếu cần


In [80]:
import re

def clean_label(text):
    match = re.search(r"\b(positive|negative|neutral)\b", text.lower())
    return match.group(1) if match else "UNKNOWN"


In [198]:
def generate_strategy(prompt_base, strategy_index):
    prompt = prompt_base + f"Strategy {strategy_index + 1}:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(
        **inputs,
        max_length=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [207]:
acc = 0
for i in range(200):
    prompt_base = zero_shot_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(3)]
    labels = extract_labels_from_thoughts("\n".join(strategies))
    if labels:
        prompt = followup_prompt(", ".join(labels))
        input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
        output = model.generate(
            **input_ids,
            max_length=20,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        final_label = tokenizer.decode(output[0], skip_special_tokens=True)
        final_label = clean_label(final_label)
        if final_label == data[i]['output']:
            acc = acc+1
print(acc/200)

0.66


In [206]:
acc = 0
for i in range(200):
    prompt_base = zero_shot_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(5)]
    labels = extract_labels_from_thoughts("\n".join(strategies))
    if labels:
        prompt = followup_prompt(", ".join(labels))
        input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
        output = model.generate(
            **input_ids,
            max_length=20,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        final_label = tokenizer.decode(output[0], skip_special_tokens=True)
        final_label = clean_label(final_label)
        if final_label == data[i]['output']:
            acc = acc+1
print(acc/200)

0.6


In [208]:
acc = 0
for i in range(200):
    prompt_base = zero_shot_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(10)]
    labels = extract_labels_from_thoughts("\n".join(strategies))
    if labels:
        prompt = followup_prompt(", ".join(labels))
        input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
        output = model.generate(
            **input_ids,
            max_length=20,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        final_label = tokenizer.decode(output[0], skip_special_tokens=True)
        final_label = clean_label(final_label)
        if final_label == data[i]['output']:
            acc = acc+1
print(acc/200)

0.6


Zero-shot + CoT + SC

In [98]:
from collections import Counter
def self_consistency(prompt, num_samples=5):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
        output = model.generate(
            **inputs,
            max_length=50,
            do_sample=True,           
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        outputs.append(decoded)
    return outputs
def majority_vote(outputs):
    counts = Counter(outputs)
    return counts.most_common(1)[0][0], counts

In [92]:
def zero_shot_CoT(text):
  return f"""
  You are a classifier assistant.{text['instruction']}. 
  Q: {text['input'] } A: Let's think step by step. The sentiment is """

In [99]:
prompt = zero_shot_CoT(data[0])
print(prompt)
samples = self_consistency(prompt, num_samples=5)
best_answer, all_votes = majority_vote(samples)

print("Self-Consistent Answer:", best_answer)
print("All votes:", all_votes)


  You are a classifier assistant.Classify the following text into 'positive', 'neutral', or 'negative'.. 
  Q: between 1900 and 1920 where did most of the migrants to the united states come from A: Let's think step by step. The sentiment is 
Self-Consistent Answer: neutral
All votes: Counter({'neutral': 5})


In [101]:
acc = 0
for i in range(200):
    prompt = zero_shot_CoT(data[i])
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)
    if best_answer == data[i]['output']:
        acc = acc + 1
    print(best_answer)
print(acc/200)

neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
negative
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
negative
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
negat

In [142]:
acc = 0
for i in range(200):
    prompt = zero_shot_CoT(data[i])
    samples = self_consistency(prompt, num_samples=10)
    best_answer, all_votes = majority_vote(samples)
    if best_answer == data[i]['output']:
        acc = acc + 1
print(acc/200)

0.78


**Few-shots**

Few-shot + CoT (ReAct)

In [103]:
acc = 0
results = []
for i in range(200):
  few_shots_prompt = f"""
  You're a sentiment classifier assistant. {data[i]['instruction']}, Recorrect each time you classifying with other LLM model
  Example:Although I had high hopes, this product was a huge disappointment.
  Let's think step by step.
  1. The person had high hopes.
  2. But the product was a disappointment.
  3. The overall sentiment is negative.
  Q: {data[i]['input'] } A: Let's think step by step. The sentiment is ?"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de == data[i]['output']:
    acc = acc+1
print(acc/200)

0.815


In [104]:
acc = 0
results = []
for i in range(200):
  few_shots_prompt = f"""
  You are a classifier assistant.{data[i]['instruction']}. Recorrect each time you classifying with other LLM model
  Q: hi, A: positive
  Q: Efficient and fast, A: positive
  Q: How can I print mouse coordinates in Rust GTK on a drawing area when clicked?, A: neutral
  Q: {data[i]['input'] } A: Let's think step by step. The answer is ?"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(data[i]['output'])
  print(de)
  if de == data[i]['output']:
    acc = acc+1
print(acc/200)

neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
positive
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
negative
neutral
negative
negative
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
positive
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral

*thử đảo vị trí các ví dụ*

In [107]:
acc = 0
results = []
for i in range(3, 200):
  few_shots_prompt = f"""
  You are a classifier assistant.{data[i]['instruction']}. Recorrect each time you classifying with other LLM model
  Q: How can I print mouse coordinates in Rust GTK on a drawing area when clicked?, A: neutral
  Q: I hate this chat, A: negative
  Q: Efficient and fast, A: positive
  Q: {data[i]['input'] } A: The sentiment is ?"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  if de == data[i]['output']:
    acc = acc+1
print(acc/197)

0.8071065989847716


Few-shots + CoT + SC

In [ ]:
def few_shots_CoT_SC(query):
    examples = find_top_k_tasks(query, 3)
    prompt = build_prompt(examples, query)
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)
    
    print("Self-Consistent Answer:", best_answer)
    print("All votes:", all_votes)
for i in range(50):
    few_shots_CoT_SC(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

Few-shots + Self-evaluation ToT 

In [195]:
def few_shots_ToT(text):
    return f"""
  You are a classifier assistant.{text['instruction']}. Recorrect each time you classifying with other LLM model
  Q: How can I print mouse coordinates in Rust GTK on a drawing area when clicked?, A: neutral
  Q: I hate this chat, A: negative
  Q: Efficient and fast, A: positive
  Q: {text['input'] } A: The sentiment is ?"""

In [203]:
acc = 0
for i in range(200):
    prompt_base = few_shots_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(3)]
    labels = extract_labels_from_thoughts("\n".join(strategies))
    if labels:
        prompt = followup_prompt(", ".join(labels))
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        output = model.generate(
            **input_ids,
            max_length=20,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        final_label = tokenizer.decode(output[0], skip_special_tokens=True)
        final_label = clean_label(final_label)
        if final_label == data[i]['output']:
            acc = acc + 1
print(acc/200)

0.82


In [201]:
acc = 0
for i in range(200):
    prompt_base = few_shots_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(5)]
    labels = extract_labels_from_thoughts("\n".join(strategies))
    if labels:
        prompt = followup_prompt(", ".join(labels))
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        output = model.generate(
            **input_ids,
            max_length=20,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        final_label = tokenizer.decode(output[0], skip_special_tokens=True)
        final_label = clean_label(final_label)
        if final_label == data[i]['output']:
            acc = acc + 1
print(acc/200)

0.71


In [202]:
acc = 0
for i in range(200):
    prompt_base = few_shots_ToT(data[i])
    strategies = [generate_strategy(prompt_base, j) for j in range(10)]
    labels = extract_labels_from_thoughts("\n".join(strategies))
    if labels:
        prompt = followup_prompt(", ".join(labels))
        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
        output = model.generate(
            **input_ids,
            max_length=20,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        final_label = tokenizer.decode(output[0], skip_special_tokens=True)
        final_label = clean_label(final_label)
        if final_label == data[i]['output']:
            acc = acc + 1
print(acc/200)

0.635


# QA

(hỏi đáp đơn giản)

In [213]:
clear_gpu()

**Zero-shot**

In [216]:
qa = []
with open('/kaggle/input/simple-qa/simple_qa.json', 'r', encoding='utf-8') as f:
    qa.append(json.load(f))

In [220]:
qa = qa[0]

In [223]:
qa[:3]

[{'instruction': 'Answer the question',
  'input': 'between 1900 and 1920 where did most of the migrants to the united states come from',
  'output': 'During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world.'},
 {'instruction': 'Answer the question',
  'input': 'What does inioluwa mean',
  'output': 'Inioluwa is a Nigerian name that means "God\'s gift" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God.'},
 {'instruction': 'Answer the question',
  'input': "Today let's pretend we are building a 16-bit computer.",
  'output': "Sure, I'd be happy to help you with building a 16-bit computer. What specific components or aspects of the computer would you lik

In [225]:
def zero_shot_prompt(text):
    return f"Q: {text['input']}. A: Answer the question "

In [229]:
print(qa[0]['input'])
print(qa[0]['output'])

between 1900 and 1920 where did most of the migrants to the united states come from
During the period between 1900 and 1920, most migrants to the United States came from Europe, particularly from Italy, Poland, and Russia. There were also significant numbers of immigrants from Canada, Mexico, and other parts of Latin America. In addition, there were smaller numbers of immigrants from Asia and other parts of the world.


In [227]:
input = tokenizer(zero_shot_prompt(qa[0]), return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    num_beams=5,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Mexico


In [247]:
test_error = """Q: In the early 20th century, where did most of the migrants to the united states come from
. A: Answer the question, which's country ? """

input = tokenizer(test_error, return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))


finland


moi lan mot nuoc ~

In [251]:
print(qa[1]['input'])
print(qa[1]['output'])

What does inioluwa mean
Inioluwa is a Nigerian name that means "God's gift" in Yoruba, a language spoken in Nigeria. It is typically given to a child who is seen as a blessing from God.


In [256]:
input = tokenizer(zero_shot_prompt(qa[1]), return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

(IV).


Zero-shot + CoT

In [257]:
def zero_shot_CoT(text):
    return f"Q: {text['input']}. A: Let's think step by step"

In [259]:
input = tokenizer(zero_shot_CoT(qa[0]), return_tensors='pt').to('cuda')
output = model.generate(
    **input,
    max_length = 100,
    num_beams=5,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

The majority of migrants to the United States between 1900 and 1920 came from Europe. So, the final answer is Europe.


In [ ]:
from sentence_transformers import SentenceTransformer, util


Zero-shot + CoT + SC

**Few-shots**

Few-shots + CoT

One-shot + CoT + ART

Few-shots + CoT + ART

Few-shots + CoT + SC

Few-shots + ToT

# Computation

**Zero-shot**

Zero-shot + CoT

In [ ]:
prompt = f"Q: {math[0]['input'] } A: Let's think step by step."
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs, max_length=160)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
math[5]['output']

In [ ]:
prompt = f"Q: {math[5]['input'] } A: Let's think step by step. The answer is ?"
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = model.generate(
    **inputs,
    max_length=100,
    num_beams=2,
    no_repeat_ngram_size=3,
    early_stopping=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(answer)

In [ ]:
acc = 0
results = []
for i in range(10):
  zero_shot_prompt = f"""
  You are a math assistant. When you need to calculate something, use the CALC() function.
  Q: {math[i]['input'] } A: Let's think step by step. The answer is ?"""
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )

  answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
  results.append(answer)
  target = extract_target(math[i]['output'])
  if answer == target:
    acc = acc+1


In [ ]:
for i in range(len(results)):
  print(results[i])
  print(extract_target(math[i]['output']))

In [ ]:
acc = 0
results = []
few_shots_prompt = f"""
 Q: {math[0]['input']}. Let's think step by step',
  """
inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=2,
      no_repeat_ngram_size=3,
      early_stopping=True
  )

answer =tokenizer.decode(outputs[0], skip_special_tokens=True)
results.append(answer)

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
print(answer)
print(extract_target(math[1]['output']))

In [ ]:
acc = 0
results = []
for i in range(10):
  zero_shot_prompt = f"""
  You are a math assistant. When you need to calculate something, use the CALC() function.
  Q: {math[i]['input'] } A: You need to answer follow this form: The answer: """
  inputs = tokenizer(zero_shot_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
  de = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(de)
  answer = extract_answer(de)
  results.append(answer)
  target = extract_target(math[i]['output'])
  if answer == target:
    acc = acc+1

In [ ]:
for i in range(10):
  print(math[i]['input'])
  print(math[i]['output'])

In [ ]:
clear_gpu()

Zero-shot + CoT + SC

Zero-shot + ToT

**Few-shots**

Few-shots + CoT

In [ ]:
acc  = 0
results = []
for i in range(3,20):
  few_shots_prompt = f"""
  You are a math assistant. Whenever you see a math expression, call the function CALC().
Q: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
A: Natalia sold CALC(48/2)=24 clips in May.
Natalia sold CALC(48+24)=72 clips altogether in April and May.
#### 72
Q: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
A: Weng earns $CALC(12/60)=0.2 per minute.
Working 50 minutes, she earned $CALC(0.2*50)=10,
#### 10
Q: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
A: In the beginning, Betty has only $CALC(100/2) = 50.
Betty's grandparents gave her $CALC(15*2)=30.
This means, Betty needs $CALC(100-50-30-15) = 5 more.
#### 5
Q: {math[i]['input'] } A: Let's think step by step. The answer is ?"""
  inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=True
  )

  answer = extract_answer(tokenizer.decode(outputs[0], skip_special_tokens=True))
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  results.append(answer)
  target = extract_target(math[i]['output'])
  if answer == target:
    acc = acc+1


In [ ]:
acc = 0
results = []
few_shots_prompt = f"""Q: Janet’s ducks lay 16 eggs per day.
  She eats three for breakfast every morning and bakes muffins for her friends every day with four.
  She sells the remainder at the farmers' market daily for $2 per fresh duck egg.
  How much in dollars does she make every day at the farmers' market?
 A: Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
 The answer: 18.

 Q: {math[1]['input']}. Let's think step by step',
  """
inputs = tokenizer(few_shots_prompt, return_tensors="pt").to('cuda')
outputs = model.generate(
      **inputs,
      max_length=100,
      num_beams=2,
      no_repeat_ngram_size=3,
      early_stopping=True
  )

answer =tokenizer.decode(outputs[0], skip_special_tokens=True)
results.append(answer)

In [ ]:
import re

def split_steps(text):
    text = text.replace('\n', ' ')
    math_expr = re.findall(r'[\$\d\s\+\-\*/xX=\.]+', text)

    steps = []
    for expr in math_expr:
        expr = expr.strip()
        expr = expr.replace('x', '*').replace('X', '*')  # chuẩn hóa phép nhân
        if '=' in expr and any(c.isdigit() for c in expr):
            steps.append(expr)

    return steps

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
with open("/kaggle/input/math-inference/final_train_math (1).json") as f:
    task_samples = [json.loads(line) for line in f]
new_tasks = []
for task in task_samples:
    cot, answer = split_cot_and_answer(task["output"])
    task["chain_of_thought"] = cot
    task["answer"] = answer
    new_tasks.append(task)

with open("math_with_cot.jsonl", "w") as f:
    for task in new_tasks:
        f.write(json.dumps(task) + "\n")
texts = [task['input'] for task in new_tasks]
vectorizer = TfidfVectorizer().fit(texts)
X = vectorizer.transform(texts)

In [ ]:
def find_similar_task(query):
    q = vectorizer.transform([query])
    idx = cosine_similarity(q, X).argmax()
    return task_samples[idx]

one-shot + CoT + ART

In [ ]:
def build_prompt(example, query):
    cot = "\n".join(example["chain_of_thought"])
    return f"""Here is a similar task:
            Task: {example['instruction']}
            Input: {example['input']}
            Chain of Thought:
            {cot}
            Output: {example['output']}
            
            Now solve this task:
            Input: {query}
            Chain of Thought:"""

In [ ]:
def one_shot_ART(query):
    example = find_similar_task(query)
    prompt = build_prompt(example, query)
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(
      **inputs,
      max_length=150,
      num_beams=5,
      no_repeat_ngram_size=3,
      early_stopping=False
  )
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
one_shot_ART("Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?")

In [ ]:
test_tasks= []
with open("/kaggle/input/math-inference/final_test_math (1).json") as f:
    test_tasks = [json.loads(line) for line in f]
for i in range(10):
    one_shot_ART(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

In [ ]:
def build_prompt(example, query):
    cot = "\n".join(example["chain_of_thought"])
    return f"""Here is a similar task:
            Task: {example['instruction']}
            Input: {example['input']}
            Chain of Thought:
            {cot}
            Output: {example['output']}
            Now solve this task:
            Input: {query}
            Solution:
Let's solve this step-by-step. Write your reasoning clearly. Final answer: ####"""

In [ ]:
for i in range(10):
    one_shot_ART(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

In [ ]:
def build_prompt(example, query):
    cot = "\n".join(example["chain_of_thought"])
    return f"""Here is a similar task:
            Task: {example['instruction']}
            Input: {example['input']}
            Chain of Thought:
            {cot}
            Output: {example['output']}
                        You're a math assistant
            Now solve this task:
            Input: {query}
            Solution:
Let's solve this step-by-step. Write your reasoning clearly. Recorrect itself. Final answer: ####"""

In [ ]:
for i in range(10):
    one_shot_ART(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

Few-shots + CoT + ART

In [ ]:
def build_prompt(examples, query):
    prompt = ""
    for ex in examples:
        cot = "\n".join(ex.get("chain_of_thought", [])) if "chain_of_thought" in ex else ex["output"]
        prompt += f"""Task: {ex.get('instruction', 'Math Problem')}
                    Input: {ex['input']}
                    Chain of Thought:
                    {cot}
                    Output: {ex['output']}
                    
                    """

    prompt += (
        "Now solve this task:\n"
        f"Input: {query}\n"
        "Solution:\n"
        "Let's solve this step-by-step. Write your reasoning clearly. Final answer: ####"
    )
    return prompt

In [ ]:
def find_top_k_tasks(query, k=3):
    q = vectorizer.transform([query])
    scores = cosine_similarity(q, X).flatten()
    top_indices = scores.argsort()[::-1][:k]
    return [task_samples[i] for i in top_indices]

In [ ]:
def find_top_k_tasks(query, k=3):
    q = vectorizer.transform([query])
    scores = cosine_similarity(q, X).flatten()
    top_indices = scores.argsort()[::-1][:k]
    return [task_samples[i] for i in top_indices]

In [ ]:
clear_gpu()

In [ ]:
for i in range(10):
    few_shots_ART(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

In [ ]:
clear_gpu()

In [ ]:
for i in range(10):
    few_shots_ART(test_tasks[i]['input'], 5)
    print(extract_target(test_tasks[i]['output']))

Làm sao để giải quyết được OOM: prompt dài ??? -> prompt compression

Few-shots + CoT(ReAct) + SELF-CONSITENCY (SC)

In [ ]:
def self_consistency(prompt, num_samples=5):
    outputs = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
        output = model.generate(
            **inputs,
            max_length=150,
            do_sample=True,           # sampling ON
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            num_return_sequences=1
        )
        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        print(decoded)
        outputs.append(decoded)
    return outputs
def majority_vote(outputs):
    answers = [extract_answer(out) for out in outputs]
    counts = Counter(answers)
    return counts.most_common(1)[0][0], counts

In [ ]:
from collections import Counter

query = test_tasks[0]['input']
prompt = build_prompt(find_similar_task(query), query)
print(prompt)
samples = self_consistency(prompt, num_samples=5)
best_answer, all_votes = majority_vote(samples)

print("Self-Consistent Answer:", best_answer)
print("All votes:", all_votes)

In [ ]:
def one_shots_CoT_SC(query):
    prompt = build_prompt(find_similar_task(query), query)
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)
    
    print("Self-Consistent Answer:", best_answer)
    print("All votes:", all_votes)

In [ ]:
for i in range(10):
    one_shots_CoT_SC(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

In [ ]:
def build_prompt(examples, query):
    prompt = ""
    for ex in examples:
        cot = "\n".join(ex.get("chain_of_thought", [])) if "chain_of_thought" in ex else ex["output"]
        prompt += f"""Task: {ex.get('instruction', 'Math Problem')}
                    Input: {ex['input']}
                    Chain of Thought:
                    {cot}
                    Output: {ex['output']}
                    
                    """

    prompt += (
        "Now solve this task:\n"
        f"Input: {query}\n"
        "Solution:\n"
        "Let's solve this step-by-step. Write your reasoning clearly. Final answer: ####"
    )
    return prompt


In [ ]:
def few_shots_CoT_SC(query):
    examples = find_top_k_tasks(query, 3)
    prompt = build_prompt(examples, query)
    samples = self_consistency(prompt, num_samples=5)
    best_answer, all_votes = majority_vote(samples)
    
    print("Self-Consistent Answer:", best_answer)
    print("All votes:", all_votes)

In [ ]:
for i in range(50):
    few_shots_CoT_SC(test_tasks[i]['input'])
    print(extract_target(test_tasks[i]['output']))

Few-shots + ToT

# Reasoning

In [ ]:
import json
file_path = "/kaggle/working/reasoning_entries.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = [json.loads(line.strip()) for line in f]

**Zero-shot**

Zero-shot + CoT

In [ ]:
def zero_shot_CoT(text):
    return f"Q: {text['input']}. A: Let's think step by step and explain"

In [ ]:
input = tokenizer(zero_shot_CoT(data[0]), return_tensors="pt").to('cuda')
output = model.generate(
    **input, 
    max_length=100,
    num_beams=5,
    no_repeat_ngram_size=3,
    early_stopping=False
)
print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
clear_gpu()

In [ ]:
answer = []
target = []
for i in range(len(val_data)):
    input = tokenizer(zero_shot_CoT(data[i]), return_tensors="pt").to('cuda')
    output = model.generate(
        **input, 
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=False
    )
    answer.append(tokenizer.decode(output[0], skip_special_token=True))
    target.append(val_data[i]['output'])

In [ ]:
for i in range(len(val_data)):
    print(answer[i])
    print(target[i])

In [ ]:
def zero_shot_ToT(text):
    return f"Q: {text['input']}. A:Let's think of three different strategies for this question. For each strategy, explain the main idea and how it could be implemented."
def followup_prompt(text):
 return f"Here are 3 possible situations: {text}. A: Which scenario is most likely to happen and why"


In [ ]:
input = tokenizer(zero_shot_ToT(data[0]), return_tensors="pt").to('cuda')
output = model.generate(
        **input, 
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=False
    )
answer =tokenizer.decode(output[0], skip_special_token=True)
print(answer)
followup_prompt(answer)

In [ ]:
input = tokenizer(followup_prompt(answer), return_tensors="pt").to('cuda')
output = model.generate(
        **input, 
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=False
    )
print(tokenizer.decode(output[0], skip_special_token=True))

Zero-shot + CoT + SC

Zero-shot + ToT

In [ ]:
answer_1 = []
branchs_1 = []
for i in range(len(val_data)):
    input = tokenizer(zero_shot_ToT(data[i]), return_tensors="pt").to('cuda')
    output = model.generate(
        **input, 
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=False
    )
    branch = tokenizer.decode(output[0], skip_special_token=True)
    branchs_1.append(branch)
    input = tokenizer(followup_prompt(branch), return_tensors="pt").to('cuda')
    output = model.generate(
            **input, 
            max_length=100,
            num_beams=5,
            no_repeat_ngram_size=3,
            early_stopping=False
        )
    answer = tokenizer.decode(output[0], skip_special_token=True)
    print(answer)
    answer_1.append(answer)

**Few-shots**

Few-shots + CoT

In [ ]:
def few_shots_CoT(text):
    return f"""
Q: Explain quantum computing in simple terms,
A: Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. In contrast to classical computers, which use bits to store and process information, quantum computers use quantum bits, or qubits. Because qubits can be in multiple states at once, quantum computers are able to perform many calculations simultaneously, which can make them much faster than classical computers for certain types of problems. While quantum computers are still in the early stages of development, they have the potential to revolutionize many fields, including cryptography, medicine, and materials science.

Q: why shouldn't I trust birds?,
A: There is no reason not to trust birds in general. However, wild birds may carry diseases and it's best to avoid handling them or coming into contact with their droppings. Also, some bird species, such as pigeons, can carry diseases such as histoplasmosis and cryptococcosis. So it's better to take precautions when interacting with wild birds.

Q: I have an array of DomElements in JS, how can I find the item with the largest height?
A: You can loop through the array of DomElements, and for each element, use the `offsetHeight` property to get its height. You can then compare this height to the current maximum height found so far and update the maximum height and corresponding DomElement accordingly. Here's an example implementation:
```javascript
function findMaxHeightElement(elements) {{
  let maxHeight = -Infinity;
  let maxHeightElement = null;

  for (let i = 0; i < elements.length; i++) {{
    const elementHeight = elements[i].offsetHeight;
    if (elementHeight > maxHeight) {{
      maxHeight = elementHeight;
      maxHeightElement = elements[i];
    }}
  }}

  return maxHeightElement;
}}
Q: {text['input']}
A: Let's think step by step
"""

In [ ]:
answer2 = []
target2 = []
for i in range(100):
    input = tokenizer(few_shots_CoT(val_data[i]), return_tensors="pt").to('cuda')
    output = model.generate(
        **input, 
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=False
    )
    answer2.append(tokenizer.decode(output[0], skip_special_token=True))
    target2.append(val_data[i]['output'])

In [ ]:
for i in range(100):
    print(answer2[i])

Few-shots + CoT + SC

Few-shots ToT

In [ ]:
def few_shots_ToT(text):
    return f"""
Q: Explain quantum computing in simple terms,
A: Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. In contrast to classical computers, which use bits to store and process information, quantum computers use quantum bits, or qubits. Because qubits can be in multiple states at once, quantum computers are able to perform many calculations simultaneously, which can make them much faster than classical computers for certain types of problems. While quantum computers are still in the early stages of development, they have the potential to revolutionize many fields, including cryptography, medicine, and materials science.

Q: why shouldn't I trust birds?,
A: There is no reason not to trust birds in general. However, wild birds may carry diseases and it's best to avoid handling them or coming into contact with their droppings. Also, some bird species, such as pigeons, can carry diseases such as histoplasmosis and cryptococcosis. So it's better to take precautions when interacting with wild birds.

Q: I have an array of DomElements in JS, how can I find the item with the largest height?
A: You can loop through the array of DomElements, and for each element, use the `offsetHeight` property to get its height. You can then compare this height to the current maximum height found so far and update the maximum height and corresponding DomElement accordingly. Here's an example implementation:
```javascript
function findMaxHeightElement(elements) {{
  let maxHeight = -Infinity;
  let maxHeightElement = null;

  for (let i = 0; i < elements.length; i++) {{
    const elementHeight = elements[i].offsetHeight;
    if (elementHeight > maxHeight) {{
      maxHeight = elementHeight;
      maxHeightElement = elements[i];
    }}
  }}

  return maxHeightElement;
}}
Q: {text['input']}
A:  A:Let's think of three different strategies for this question. For each strategy, explain the main idea and how it could be implemented.
"""

In [ ]:
answer_2 = []
branchs_2 = []
for i in range(len(val_data)):
    input = tokenizer(few_shots_ToT(data[i]), return_tensors="pt").to('cuda')
    output = model.generate(
        **input, 
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=3,
        early_stopping=False
    )
    branch = tokenizer.decode(output[0], skip_special_token=True)
    branchs_2.append(branch)
    input = tokenizer(followup_prompt(branch), return_tensors="pt").to('cuda')
    output = model.generate(
            **input, 
            max_length=100,
            num_beams=5,
            no_repeat_ngram_size=3,
            early_stopping=False
        )
    answer = tokenizer.decode(output[0], skip_special_token=True)
    print(answer)
    answer_2.append(answer)

# Finetune

In [ ]:
import json
classification = []
with open('classification_instruction_data_en.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        classification.append(json.loads(line.strip()))
qa = []
computation = []
reasoning = []

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
os.environ["WANDB_DISABLED"] = "true"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model2 = T5ForConditionalGeneration.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False
)
trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model2)
)
trainer.train()